# Clustering

In [192]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import kneed as kn
import plotly.graph_objects as go
import random

In [193]:
from plotly.subplots import make_subplots

#scaling, normalization
from sklearn.preprocessing import StandardScaler, MinMaxScaler

#kmeans, dbscan, hierarchical (sklearn)
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
#evaluation
from sklearn.metrics import silhouette_score

#distance matrix (dbscan elbow, hierarchical)
from scipy.spatial.distance import pdist, squareform
from scipy.stats import pearsonr

# hierarchical (scipy)
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster

from kneed import KneeLocator

In [194]:
from sklearn.metrics import *
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering
from sklearn.neighbors import kneighbors_graph

In [195]:


#DATI SENZA (POCHI) OUTLIERS E SENZA VARIABILI IRRILEVANTI E AGGREGATE LE REVIEWS
#train_cleaned_v.2 file dati log e norm
#train_log_unscaled file dati log non norm
#train_cleaned_v.1 file dati non log e non norm

df = pd.read_csv('train_nolog.csv')

In [196]:
df.head()

,originalTitle,rating,startYear,runtimeMinutes,awardWins,numVotes,totalImages,totalVideos,totalCredits,titleType,AwNmExWins,canHaveEpisodes,isAdult,numRegions,countryOfOrigin,genres,ratingMean,ReviewsTotal
0,Neogenic Nightmare Chapter 3: Hydro-Man,"(7, 8]",1995,40,0,779,1,0,21,tvEpisode,0,False,False,1,['US'],"Action,Adventure,Animation",7.5,1
1,Looping,"(5, 6]",1982,28,0,11,1,0,1,videoGame,0,False,False,1,['XWG'],Action,5.5,0
2,Idealnaya para,"(5, 6]",1992,92,0,38,1,0,24,movie,0,False,False,1,['RU'],Comedy,5.5,1
3,MasterChef Celebrity México,"(5, 6]",2021,53,0,15,22,0,56,tvSeries,0,True,False,1,['MX'],Reality-TV,5.5,0
4,Seutateueob,"(7, 8]",2020,80,1,15718,347,2,47,tvSeries,1,True,False,32,['KR'],"Comedy,Drama,Romance",7.5,456


#### Fill missing values and convert to numerical (where possible)

In [197]:
df.dtypes

originalTitle       object
rating              object
startYear            int64
runtimeMinutes       int64
awardWins            int64
numVotes             int64
totalImages          int64
totalVideos          int64
totalCredits         int64
titleType           object
AwNmExWins           int64
canHaveEpisodes       bool
isAdult               bool
numRegions           int64
countryOfOrigin     object
genres              object
ratingMean         float64
ReviewsTotal         int64
dtype: object

In [198]:
del df['originalTitle']
del df['rating']
del df['titleType']
del df['countryOfOrigin']
del df['genres']
del df['isAdult']
del df['canHaveEpisodes']

In [199]:
df.head()

,startYear,runtimeMinutes,awardWins,numVotes,totalImages,totalVideos,totalCredits,AwNmExWins,numRegions,ratingMean,ReviewsTotal
0,1995,40,0,779,1,0,21,0,1,7.5,1
1,1982,28,0,11,1,0,1,0,1,5.5,0
2,1992,92,0,38,1,0,24,0,1,5.5,1
3,2021,53,0,15,22,0,56,0,1,5.5,0
4,2020,80,1,15718,347,2,47,1,32,7.5,456


In [200]:
df.describe()

,startYear,runtimeMinutes,awardWins,numVotes,totalImages,totalVideos,totalCredits,AwNmExWins,numRegions,ratingMean,ReviewsTotal
count,15925.000000,15925.000000,15925.000000,15925.000000,15925.000000,15925.000000,15925.000000,15925.000000,15925.000000,15925.000000,15925.000000
mean,1992.215636,59.348069,0.410235,1413.813375,11.254066,0.261852,59.784364,0.547441,3.595290,6.672245,9.678493
std,25.923192,39.015371,2.593600,18146.149450,67.679200,2.970572,95.401646,3.565226,5.886415,1.377651,59.385049
min,1878.000000,0.000000,0.000000,5.000000,0.000000,0.000000,1.000000,0.000000,1.000000,0.500000,0.000000
25%,1978.000000,29.000000,0.000000,15.000000,1.000000,0.000000,17.000000,0.000000,1.000000,5.500000,0.000000
50%,1997.000000,53.000000,0.000000,38.000000,2.000000,0.000000,35.000000,0.000000,1.000000,6.500000,1.000000
75%,2013.000000,90.000000,0.000000,156.000000,6.000000,0.000000,66.000000,0.000000,4.000000,7.500000,3.000000
max,2024.000000,570.000000,145.000000,825934.000000,3504.000000,258.000000,1885.000000,124.000000,69.000000,9.500000,1772.000000


In [ ]:
columns = df.columns

In [ ]:
#NORMALIZZAZIONE MINMAX O ZSCORE, COMMMENTARE UNA DELLE DUE RIGHE IN BASE AL TIPO DI NORMALIZZAZIONE CHE SI VUOLE UTILIZZARE
#PER I NOSTRI DATI è MEGLIO MIN/MAX, Z-SCORE FA CAGARE PERCHÉ I DATI NON SONO DISTRIBUITI NORMALMENTE (SCRIVERLO NEL REPORT)

scaler = MinMaxScaler()
#scaler = StandardScaler()
train_data = scaler.fit_transform(df)
train_data_df = pd.DataFrame(train_data, columns=columns)
train_data_df.describe()

In [ ]:
%%time
sse_list = []
sil_list = []
k_values = range(2, 21)

for k in k_values:
    kmeans = KMeans(init='k-means++', n_clusters=k, n_init=10, max_iter=100)
    kmeans.fit(train_data)
    sse_list.append(kmeans.inertia_)
    sil_list.append(silhouette_score(train_data, kmeans.labels_))

kn = KneeLocator(k_values, sse_list, curve='convex', direction='decreasing')
elbow_k = kn.elbow
print("Numero ottimale di cluster (k) secondo il metodo del gomito:", elbow_k)

In [ ]:


# Crea la figura con due sottotrame (subplots)
fig = make_subplots(rows=1, cols=2)

# Grafico SSE vs k
fig.add_trace(
    go.Scatter(x = [x + 2 for x in range(len(sse_list))], y=sse_list, line=dict(color='#FFA500'), mode='lines+markers', name="SSE"),
    row=1, col=1
)
fig.update_xaxes(title_text="k", row=1, col=1)
fig.update_yaxes(title_text="SSE", row=1, col=1)

# Grafico Silhouette Score vs k
fig.add_trace(
    go.Scatter(x = [x + 2 for x in range(len(sil_list))], y=sil_list, line=dict(color='black'), mode='lines+markers', name="Silhouette"),
    row=1, col=2
)
fig.update_xaxes(title_text="k", row=1, col=2)
fig.update_yaxes(title_text="Silhouette Score", row=1, col=2)

# Layout generale della figura
fig.update_layout(title_text="OD: SSE and Silhouette Score",
                  showlegend=False, title_x=0.5,font=dict(
        family="Arial",  # Font family
        size=20,         # Font size for all text elements
        color="Black"
                      ))    # Font color for all text elements)

# Mostra il grafico
fig.show()

In [ ]:
kmeans = KMeans(init='k-means++', n_clusters=8, n_init=1000, max_iter=100)
kmeans.fit(train_data)

In [ ]:
print('centroids', kmeans.cluster_centers_)
print('labels', np.unique(kmeans.labels_, return_counts=True))
print('sse', kmeans.inertia_)
print('silhouette', silhouette_score(train_data, kmeans.labels_))

In [ ]:
'''
df['kmeans_labels'] = kmeans.labels_
df['kmeans_labels'] = df['kmeans_labels'].map({0:"A", 1: "B", 2: "C"})

sns.scatterplot(data=df, 
                x="startYear", 
                y="ratingCount", 
                hue='kmeans_labels', 
                style='kmeans_labels',
                palette="bright", 
                hue_order=["A", "B", "C"]
               )
plt.legend()
plt.scatter(centers[:,0], centers[:,2], c='red', marker='*', s=200)
plt.show()
'''

In [ ]:
n_points = train_data.shape[0]
#generate distance matrix
distance_matrix = squareform(pdist(train_data, metric='euclidean')) 

#Otteniamo le etichette di cluster dal modello k-means
labels = kmeans.labels_

#generate ideal similarity matrix
similarity_matrix = np.equal.outer(labels, labels).astype(int)


# Convertiamo la matrice delle distanze in una matrice di similarità
# (simply similarity_matrix = max_distance - distance_matrix)
max_distance = np.max(distance_matrix)
similarity_from_distance = distance_matrix - max_distance 

'''
# Rendi entrambe le matrici 1D per il calcolo della correlazione
similarity_from_distance_flat = similarity_from_distance.flatten()
similarity_matrix_flat = similarity_matrix.flatten()

# Calcola la correlazione di Pearson
correlation, _ = pearsonr(similarity_from_distance_flat, similarity_matrix_flat)

print("Correlation between distance and ideal similarity matrices:", correlation)
'''
# Selezioniamo solo la parte inferiore della matrice, inclusa la diagonale
lower_triangle_indices = np.tril_indices(n_points)

# Appiattiamo solo la parte inferiore della matrice
similarity_from_distance_flat = similarity_from_distance[lower_triangle_indices]
similarity_matrix_flat = similarity_matrix[lower_triangle_indices]

# Calcolare la correlazione di Pearson tra i vettori appiattiti
correlation, _ = pearsonr(similarity_from_distance_flat, similarity_matrix_flat)

print("Correlation between distance and ideal similarity matrices:", correlation)

In [ ]:
import plotly.graph_objects as go

# Crea una figura vuota
fig = go.Figure()

# Aggiungi una linea per ogni centroide del cluster
for i, center in enumerate(kmeans.cluster_centers_):
    fig.add_trace(go.Scatter(
        x=list(df.columns),
        y=center,
        mode='lines+markers',
        name=f'Cluster {i}',
        line=dict(width=3)
    ))

# Imposta il titolo e le etichette
fig.update_layout(
    title="Centroid values for each feature across clusters",
    xaxis_title="Features",
    yaxis_title="Centroid values",
    legend=dict(x=1, y=1),
    width=800, height=400
)

# Aggiungi la griglia all'asse y
fig.update_yaxes(showgrid=True)
fig.update_layout(title_x=0.5)

# Mostra il grafico
fig.show()


Guardando questo grafico si possono elimare le variabili dove i gruppi sono simili tra loro

PROVA CONFRONTO DATI RANDOMIZZATI

In [ ]:
def randomize_data(train_data):
    """Randomizza l'ordine delle righe del dataset"""
    return np.random.permutation(train_data)

# Calcola il clustering per i dati randomizzati
def kmeans_on_random_data(train_data, k):
    X_randomized = randomize_data(train_data)
    kmeans_random = KMeans(n_clusters=k, random_state=42)
    kmeans_random.fit(X_randomized)
    sse_random = kmeans_random.inertia_
    silhouette_random = silhouette_score(X_randomized, kmeans_random.labels_)
    return sse_random, silhouette_random

In [ ]:
n_permutations = 100  # Numero di permutazioni per SSE
sse_random_results = []
silhouette_random_results = []

for _ in range(n_permutations):
    sse_random, silhouette_random = kmeans_on_random_data(train_data, k)
    sse_random_results.append(sse_random)
    silhouette_random_results.append(silhouette_random)

In [ ]:
from statistics import mean

mean_sse_random = mean(sse_random_results)
mean_silhouette_random = mean(silhouette_random_results)

print(mean_sse_random)
print(mean_silhouette_random)

In [ ]:
sse_real = kmeans.inertia_
silhouette_real = silhouette_score(train_data, kmeans.labels_)
# Visualizza i risultati di SSE per i dati reali e randomizzati
plt.figure(figsize=(10, 10))

# Plotta gli histogrammi per SSE
plt.subplot(2, 1, 1)
sns.histplot(sse_random_results, kde=True, label='Randomized SSE', color='blue', bins=10)
plt.axvline(x=sse_real, color='red', linestyle='--', label='Real SSE')
plt.legend()
plt.xlabel('SSE',fontsize=13)
plt.ylabel('Density',fontsize=13)
plt.title('SSE Comparison',fontsize=13)

# Plotta gli histogrammi per Silhouette
plt.subplot(2, 1, 2)
sns.histplot(silhouette_random_results, kde=True, label='Randomized Silhouette', color='blue', bins=30)
plt.axvline(x=silhouette_real, color='red', linestyle='--', label='Real Silhouette')
plt.legend()
plt.xlabel('Silhouette Score',fontsize=13)
plt.ylabel('Density',fontsize=13)
plt.title('Silhouette Score Comparison',fontsize=13)

plt.tight_layout()
plt.show()

FILE TRAIN_CLEANED_V1
  
SCELTO K=5  
CORRELAZIONE -0.49  
SSE 460  
SILHOUETTE SCORE 0.3

-------------   
FILE TRAIN_CLEANED_V2  
  
K=7  
CORRELAIONE -0.39  
SSE 1588  
SILHOUETTE SCORE 0.16  

-------  

SELEZIONATE VARIABILI NUMVOTES, NUMREG TOTALIMAG E REVIEWSTOTAL  (da file v.2)  
k=5  
CORRELAZIONE -0.53  
sse 304  
silhouette 0.38  

#### CUT DATASET 

In [201]:
df_cut = df[['numVotes', 'numRegions', 'ReviewsTotal']]

In [202]:
df_cut.head()

,numVotes,numRegions,ReviewsTotal
0,779,1,1
1,11,1,0
2,38,1,1
3,15,1,0
4,15718,32,456


In [203]:
columns = df_cut.columns

scaler = MinMaxScaler()
train_data = scaler.fit_transform(df_cut)
train_data_df = pd.DataFrame(train_data, columns=columns)
train_data_df.describe()

,numVotes,numRegions,ReviewsTotal
count,15925.000000,15925.000000,15925.000000
mean,0.001706,0.038166,0.005462
std,0.021971,0.086565,0.033513
min,0.000000,0.000000,0.000000
25%,0.000012,0.000000,0.000000
50%,0.000040,0.000000,0.000564
75%,0.000183,0.044118,0.001693
max,1.000000,1.000000,1.000000


In [206]:
%%time
sse_list = []
sil_list = []
k_values = range(2, 21)

for k in k_values:
    kmeans = KMeans(init='k-means++', n_clusters=k, n_init=10, max_iter=100)
    kmeans.fit(train_data)
    sse_list.append(kmeans.inertia_)
    sil_list.append(silhouette_score(train_data, kmeans.labels_))

kn = KneeLocator(k_values, sse_list, curve='convex', direction='decreasing')
elbow_k = kn.elbow
print("Numero ottimale di cluster (k) secondo il metodo del gomito:", elbow_k)

Numero ottimale di cluster (k) secondo il metodo del gomito: 6
CPU times: total: 2min 49s
Wall time: 1min 37s


In [207]:
# Crea la figura con due sottotrame (subplots)
fig = make_subplots(rows=1, cols=2)

# Grafico SSE vs k
fig.add_trace(
    go.Scatter(x = [x + 2 for x in range(len(sse_list))], y=sse_list, line=dict(color='#FFA500'), mode='lines+markers', name="SSE"),
    row=1, col=1
)
fig.update_xaxes(title_text="k", row=1, col=1)
fig.update_yaxes(title_text="SSE", row=1, col=1)

# Grafico Silhouette Score vs k
fig.add_trace(
    go.Scatter(x = [x + 2 for x in range(len(sil_list))], y=sil_list, line=dict(color='black'), mode='lines+markers', name="Silhouette"),
    row=1, col=2
)
fig.update_xaxes(title_text="k", row=1, col=2)
fig.update_yaxes(title_text="Silhouette Score", row=1, col=2)

# Layout generale della figura
fig.update_layout(title_text="OD_CUT: SSE and Silhouette Score",
                  showlegend=False, title_x=0.5,font=dict(
        family="Arial",  # Font family
        size=20,         # Font size for all text elements
        color="Black"
                      ))    # Font color for all text elements)

# Mostra il grafico
fig.show()

# Mostra il valore dell'SSE per k=6
# print(sse_list[4]) # Corrisponde a k = 6, dato che la lista parte da 0


In [ ]:
kmeans = KMeans(init='k-means++', n_clusters=6, n_init=1000, max_iter=100)
kmeans.fit(train_data)

In [ ]:
print('centroids', kmeans.cluster_centers_)
print('labels', np.unique(kmeans.labels_, return_counts=True))
print('sse', kmeans.inertia_)
print('silhouette', silhouette_score(train_data, kmeans.labels_))

In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

# Supponiamo che 'df' sia il tuo DataFrame e 'variable_3' sia la terza variabile
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

# Crea il grafico 3D
scatter = ax.scatter(df_cut['numVotes'], df_cut['numRegions'], df_cut['ReviewsTotal'], 
                     c=kmeans.labels_, cmap='viridis')

# Aggiungi etichette agli assi
ax.set_xlabel('NumVotes')
ax.set_ylabel('numRegions')
ax.set_zlabel('ReviewsTotal')

# Aggiungi una legenda
legend1 = ax.legend(*scatter.legend_elements(), title="Labels")
ax.add_artist(legend1)

ax.set_box_aspect([1, 1, 1])
ax.set_zlabel('ReviewsTotal', labelpad=0)
# Mostra il grafico
plt.show()

In [ ]:
n_points = train_data.shape[0]
#generate distance matrix
distance_matrix = squareform(pdist(train_data, metric='euclidean')) 

#Otteniamo le etichette di cluster dal modello k-means
labels = kmeans.labels_

#generate ideal similarity matrix
similarity_matrix = np.equal.outer(labels, labels).astype(int)


# Convertiamo la matrice delle distanze in una matrice di similarità
# (simply similarity_matrix = max_distance - distance_matrix)
max_distance = np.max(distance_matrix)
similarity_from_distance = distance_matrix - max_distance 

'''
# Rendi entrambe le matrici 1D per il calcolo della correlazione
similarity_from_distance_flat = similarity_from_distance.flatten()
similarity_matrix_flat = similarity_matrix.flatten()

# Calcola la correlazione di Pearson
correlation, _ = pearsonr(similarity_from_distance_flat, similarity_matrix_flat)

print("Correlation between distance and ideal similarity matrices:", correlation)
'''
# Selezioniamo solo la parte inferiore della matrice, inclusa la diagonale
lower_triangle_indices = np.tril_indices(n_points)

# Appiattiamo solo la parte inferiore della matrice
similarity_from_distance_flat = similarity_from_distance[lower_triangle_indices]
similarity_matrix_flat = similarity_matrix[lower_triangle_indices]

# Calcolare la correlazione di Pearson tra i vettori appiattiti
correlation, _ = pearsonr(similarity_from_distance_flat, similarity_matrix_flat)

print("Correlation between distance and ideal similarity matrices:", correlation)

In [ ]:
import plotly.graph_objects as go

# Crea una figura vuota
fig = go.Figure()

# Aggiungi una linea per ogni centroide del cluster
for i, center in enumerate(kmeans.cluster_centers_):
    fig.add_trace(go.Scatter(
        x=list(df_cut.columns),
        y=center,
        mode='lines+markers',
        name=f'Cluster {i}',
        line=dict(width=3)
    ))

# Imposta il titolo e le etichette
fig.update_layout(
    title="Centroid values for each feature across clusters",
    xaxis_title="Features",
    yaxis_title="Centroid values",
    legend=dict(x=1, y=1),
    width=800, height=400
)

# Aggiungi la griglia all'asse y
fig.update_yaxes(showgrid=True)
fig.update_layout(title_x=0.5)

# Mostra il grafico
fig.show()

In [ ]:
def randomize_data(train_data):
    """Randomizza l'ordine delle righe del dataset"""
    return np.random.permutation(train_data)

# Calcola il clustering per i dati randomizzati
def kmeans_on_random_data(train_data, k):
    X_randomized = randomize_data(train_data)
    kmeans_random = KMeans(n_clusters=k, random_state=42)
    kmeans_random.fit(X_randomized)
    sse_random = kmeans_random.inertia_
    silhouette_random = silhouette_score(X_randomized, kmeans_random.labels_)
    return sse_random, silhouette_random

In [ ]:
n_permutations = 100  # Numero di permutazioni per SSE
sse_random_results = []
silhouette_random_results = []

for _ in range(n_permutations):
    sse_random, silhouette_random = kmeans_on_random_data(train_data, k)
    sse_random_results.append(sse_random)
    silhouette_random_results.append(silhouette_random)

In [ ]:
from statistics import mean

mean_sse_random = mean(sse_random_results)
mean_silhouette_random = mean(silhouette_random_results)

print(mean_sse_random)
print(mean_silhouette_random)

In [ ]:
sse_real = kmeans.inertia_
silhouette_real = silhouette_score(train_data, kmeans.labels_)
# Visualizza i risultati di SSE per i dati reali e randomizzati
plt.figure(figsize=(10, 10))

# Plotta gli histogrammi per SSE
plt.subplot(2, 1, 1)
sns.histplot(sse_random_results, kde=True, label='Randomized SSE', color='blue', bins=10)
plt.axvline(x=sse_real, color='red', linestyle='--', label='Real SSE')
plt.legend()
plt.xlabel('SSE',fontsize=13)
plt.ylabel('Density',fontsize=13)
plt.title('SSE Comparison',fontsize=13)

# Plotta gli histogrammi per Silhouette
plt.subplot(2, 1, 2)
sns.histplot(silhouette_random_results, kde=True, label='Randomized Silhouette', color='blue', bins=30)
plt.axvline(x=silhouette_real, color='red', linestyle='--', label='Real Silhouette')
plt.legend()
plt.xlabel('Silhouette Score',fontsize=13)
plt.ylabel('Density',fontsize=13)
plt.title('Silhouette Score Comparison',fontsize=13)

plt.tight_layout()
plt.show()

# Bisecting K-Means

In [ ]:
from sklearn.cluster import BisectingKMeans

In [ ]:
bkmeans = BisectingKMeans(n_clusters=4)
bkmeans.fit(train_data_df)

In [ ]:
sse = bkmeans.inertia_
print("SSE:", sse)
silhouette_avg = silhouette_score(train_data_df, bkmeans.labels_)
print("Silhouette coefficient:", silhouette_avg)

In [ ]:
import plotly.express as px

# Crea il grafico a dispersione interattivo
fig = px.scatter(
    train_data_df, 
    x="numRegions", 
    y="numVotes", 
    color=bkmeans.labels_.astype(str),  # Converti i label a stringa per usarli come categorie
    symbol=bkmeans.labels_.astype(str), # Usa simboli diversi per ogni cluster
    color_discrete_sequence=px.colors.qualitative.Bold,  # Palette colori "bright"
    title="Cluster Visualization with Plotly",
    labels={'startYear': 'startYear', 'popularityIndex': 'Popularity Index', 'color': 'Cluster'},
)

# Mostra il grafico
fig.show()


### Silhouette bisecting k-means


In [ ]:
from yellowbrick.cluster import SilhouetteVisualizer

In [ ]:

colors=['C0', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9']
n_clust = 8
fig, axs = plt.subplots(2,n_clust-2, figsize=(15,10))
for i in range(2, n_clust):
    bkmeans = BisectingKMeans(n_clusters=i)
    visualizer = SilhouetteVisualizer(bkmeans, colors=colors, ax=axs[0][i-2])
    axs[0][i-2].set_ylabel("k=" + str(i), fontsize=12)
    axs[0][i-2].set_xlabel("Silhouette Coefficient", fontsize=12)
    visualizer.fit(train_data_df)
    sns.scatterplot(data=train_data_df, x="numRegions", y="numVotes", hue=bkmeans.labels_, 
                    palette=sns.color_palette(colors[:i]), style=bkmeans.labels_, ax=axs[1][i-2])
plt.tight_layout()



In [ ]:
silhouette_score(train_data_df, bkmeans.labels_)

### Analysis by centroid-based methods

- K-Means (mandatory), Bisecting K-Means (optional), X-Means (optional);

- Choice the attributes, identify the best value of k, discuss the clusters.

### Analysis by density-based clustering

- DBSCAN (mandatory), OPTICS (optional);

- Choice the attributes, identify the best parameter configuration, discuss clusters.

### Analysis by hierarchical clustering

- Choice the attributes, the distance function, analyse several dendrograms.

### Final discussion

- Which is the best algorithm? Remember that best is studied w.r.t. several aggregate 
statistics, cluster distributions and w.r.t. the typology of algorithm used for that 
particular dataset;
